<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/ex11notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML ex11notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?ML/2023)


----
## 教師なし学習とは
----




---
### ［復習］教師あり学習と教師なし学習





これまで見てきたように，機械学習で扱う問題は，いずれも，「何らかのデータ $\boldsymbol{x}$ を入力すると適当な値 $\boldsymbol{y}$ を出力する」という情報処理となっています．
以下の左の図は，この処理を行う仕組みを $f$ という記号で表したものです．右の方は，$f$ を関数として数式で表現したものです．

$$
\boldsymbol{y} \longleftarrow \fbox{$f$} \longleftarrow \boldsymbol{x} \qquad\qquad\qquad\qquad \boldsymbol{y} = f(\boldsymbol{x})
$$

この $f$ のことを，**学習モデル**（または単に**モデル**(model)）といいます．
モデルには調節可能な **変数**／**パラメータ**(parameter)があり，その値を調節することで，入力に対するモデルの出力を変化させることができます．

機械学習の問題では，モデルの「望ましい出力」が予め定められており，モデル出力がそれに近づくようにパラメータを調節します．
この調節は，人手ではなく自動的に行われます．
この過程を**学習**（learning, または **訓練** (training)）といいます．
学習の際には，様々なデータをモデルへ入力して，その出力の値に応じてパラメータを調節していきます．
その際に用いるデータを**学習データ**（learning data, 訓練データ(training data)）といいます．







学習の手法には様々なものがありますが，次の二つに大別することができます（注）．

**教師あり学習** (supervised learning): 個々の学習データが，「入力」とそれに対する「出力の正解」のペアとして与えられる．


**教師なし学習** (unsupervised learning): 学習データは入力のみで構成され，出力の正解は与えられない．

<span style="font-size: 75%">
※注: もう一つ，強化学習(reinforcement learning)というのもあるのですが，この授業では説明しません．
</span>



これまでのこの授業では，主に教師あり学習を扱っていました．教師あり学習の問題を「回帰」と「識別」の二つに大きく分けて，様々な学習アルゴリズムを説明してきました．
いずれの場合でも，学習データは，「入力」と「それに対する出力の正解」のペアとして与えられており，モデル出力を正解に近づけるように学習が行われるのでした．

---
### 教師なし学習とは


**教師なし学習**(unsupervised learning)の目的は，大量のデータが与えられたときに，そのデータのもつ規則性や構造を見つけ出すことです．
どのような規則性や構造を持っているかは事前には分かっていません．それらを知り，データから有用な情報を抽出することを目指します．
同様のことを目的とする技術や研究分野に**データマイニング**(data mining，注)というものがあり，両者は大きく重なっています．

教師あり学習では出力の正解が学習データとして与えられましたが，
教師なし学習では，学習データは入力となるもののみが与えられます．出力の正解は与えられません．
データからどのような規則性や構造を見出したいかによって様々な学習アルゴリズムがあり，出力がどうなるかはアルゴリズムによります．

<span style="font-size: 75%">
※注: 英語の名詞 mine には「鉱山」や「資源」といった意味があり，動詞 mine は「（鉱物資源を）採掘する」といった意味があります．Minecraft の mine は...．
</span>

教師あり学習と教師なし学習の違いについて理解を深めるため，様々な動物の画像を分類する問題を例にあげます．
これは，教師ありの設定では「識別」の問題であり，教師なしの設定では「クラスタリング」の問題として扱えます（「クラスタリング」については詳しくは後述）．

<dl>
<dt>識別の問題として</dt>
<dd>例えば，「イヌ，ネコ,ヒト，カピバラの4種」や「もふもふなのとそうでないのの2種」のように，分類の仕方をあらかじめ決める．学習データには，入力となる画像だけでなく，その画像の写った動物がどの分類のものであるかの情報が含まれている．学習することで，未分類の画像も正しく分類できるようになることが目的．</dd>
<dt>クラスタリングの問題として</dt>
<dd>どのように分類されるかは学習させてみるまで分からない．学習データは入力となる画像のみで，どう分類すべきかを表す情報は与えられていない．学習によって得られた結果を分析して，どんな画像が（写った動物が）どのように分類されるかを理解することが目的．</dd>
</dl>

---
### クラスタリングと次元削減



教師なしの学習アルゴリズムには様々なものがありますが，この授業では，代表例として，**クラスタリング**(clustering)と**次元削減**(次元圧縮とも，dimensionality reduction)の二種を取り上げます．
以下，$N$個の$D$次元ベクトルから成るデータ集合
$$
\{ \mathbf{x}_n \in {\cal R}^{D} | n = 1, 2, \ldots, N\}
$$
を学習データとした場合について，クラスタリングと次元削減がそれぞれどのようなものか説明します．

<img src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/clustering_dimreduction.png">

#### クラスタリング

**クラスタリング**(clustering)は，大量のデータが与えられたときに，似たようなデータを集めていくつかの「塊」に分ける手法です．「塊」のことを**クラスタ**(cluster)といいます．
クラスタリングを行うことで，データの構造を把握（いくつのクラスタに分けられるのかを知る，どのデータとどのデータが似ているのかを知る，等）したり，大量のデータを要約（クラスタごとにデータの性質を記述する，等）したりすることができます．

例1: 学生たちの20科目の成績を表すデータ（$D=20$）をクラスタリング．

クラスタリングによって，例えば，3つのクラスタに分けられることが分かったならば...．クラスタごとに成績の傾向が異なるので，学習指導の仕方をそれに応じて変えるようなことが可能となる．




例2: ネットショッピングの買い物傾向を表すデータをクラスタリング．

ネットショッピングサイトの経営者が，顧客がどんな商品を購入したかのデータをクラスタリングして，顧客を買い物傾向によってグループ分けできたならば...．
顧客の傾向に応じてサイトに表示させる広告やおすすめ商品の表示を変えることができる（注）．

<span style="font-size: 75%">
※注: ここでは直感的な例としてこのような問題設定をあげましたが，実際には，この問題は,クラスタリングとは少し設定が異なるところがあり，「協調フィルタリング」という問題として扱われています．
</span>

例3: 遺伝子データをクラスタリング

様々な生物種の遺伝子データ（DNA配列など）をクラスタリングして解析できたならば...．
生物種の間の類縁関係を推定（種Aと種Bは同じクラスタに分類されるので，同じ種から進化した可能性が高いと考える，等）したりできるかもしれない．
ウイルスの変異を分析したりも...．

#### 次元削減

**次元削減**(次元圧縮とも，dimensionality reduction)は，その名の通り，データの「次元数」(dimensionality)を減らす手法です．
$D$個の特徴量から成る $D$ 次元ベクトル $\mathbf{x}$ がデータとして与えられた場合，これを $H$ 次元（$H<D$）のベクトル $\mathbf{y}$ に変換します．
変換後のデータは元データよりも少数の特徴で表されているため，その後の分析がより簡単になります．
また，データを処理する際に必要な記憶容量や計算量の削減にもつながります．

例1: 学生たちの20科目の成績を表すデータ（$D=20$）を次元削減．

例えば，元データの特徴をよくとらえた3次元のデータに変換できたならば...．3つの特徴量の値によって一人ひとりの学生の傾向を分析することができる．次元を減らすことで分析や可視化が容易になる．

例2: 様々なひとの顔画像（幅100画素高さ100画素のRGBカラー画像，$D = 30000$）を次元削減．

元画像の本質的な特徴をなるべく保ったままで30次元のデータに変換できたならば...．ネットワーク越しに顔画像のデータを送信するとして，3万個の画素値を送るかわりに，30個の数値を送るだけで済ませられるかも？
